# Using WAVE to secure a model

This notebook will set up the WAVE context for the integration demo. We assume that there are two parties: the one training the model, and the one using it.

First, we connect to wave:

In [1]:
import wave3 as wv
import grpc

# create the connection to wave
wave = wv.WAVEStub(grpc.insecure_channel("localhost:410"))

Now we create a WAVE entity that represents the organization namespace:

In [6]:
orgNamespaceEntity = wave.CreateEntity(wv.CreateEntityParams())
_ = wave.PublishEntity(wv.PublishEntityParams(DER=orgNamespaceEntity.PublicDER))

Now, separately, the party receving the model would create their own entity

In [7]:
recipientEntity = wave.CreateEntity(wv.CreateEntityParams())
_ = wave.PublishEntity(wv.PublishEntityParams(DER=recipientEntity.PublicDER))

Finally, the organization would grant the ability to decrypt certain categories of models to the recipient

In [9]:
_ = wave.CreateAttestation(wv.CreateAttestationParams(
    perspective=wv.Perspective(
        entitySecret=wv.EntitySecret(DER=orgNamespaceEntity.SecretDER)),
    subjectHash=recipientEntity.hash,
    publish=True,
    policy=wv.Policy(rTreePolicy=wv.RTreePolicy(
        namespace=orgNamespaceEntity.hash,
        indirections=5,
        statements=[
            wv.RTreePolicyStatement(
                permissionSet=wv.WaveBuiltinPSET,
                permissions=[wv.WaveBuiltinE2EE],
                resource="models/pong",
            )]
    ))))